### Include registration modules

In [1]:
import copy

from functools import partial
from time import time

from collections.abc import Callable

import numpy as np

import open3d
import open3d.geometry as geom
import open3d.pipelines.registration as reg
import open3d.visualization as vis
import open3d.utility as util

from mynd.registration import (
    MultiTargetIndex,
    generate_cascade_indices,
)

from mynd.registration import (
    downsample_point_cloud,
    estimate_point_cloud_normals,
)

from mynd.registration import (
    register_icp,
    register_colored_icp,
    build_pose_graph,
    optimize_pose_graph,
)

from mynd.spatial import decompose_transformation

from mynd.visualization import (
    visualize_registration,
    create_subplots,
    trace_registration_result,
)

from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


### Load environment and configure data loaders

In [2]:
from pathlib import Path

from mynd.geometry import PointCloud, PointCloudLoader
from mynd.io import read_point_cloud, create_point_cloud_loader

DATA_DIR: Path = Path("/home/martin/dev/mynd/.cache")

point_cloud_files: dict = {
    0: DATA_DIR / Path("qdc5ghs3_20100430_024508.ply"),
    1: DATA_DIR / Path("qdc5ghs3_20120501_033336.ply"),
    2: DATA_DIR / Path("qdc5ghs3_20130405_103429.ply"),
    3: DATA_DIR / Path("qdc5ghs3_20210315_230947.ply"),
}

loaders: dict[int, PointCloudLoader] = {
    key: create_point_cloud_loader(path)
    for key, path in point_cloud_files.items()
    if path.exists()
}

count = len(loaders)
if count < 2:
    logger.error(f"invalid number of point clouds for registration: {count}")

### Logging Helpers

In [3]:
from mynd.registration import RegistrationResult


def log_registration(
    source: int, target: int, result: RegistrationResult
) -> None:
    """TODO"""

    scale, rotation, translation = decompose_transformation(
        result.transformation
    )

    logger.info("")
    logger.info(f"Source:       {source}")
    logger.info(f"Target:       {target}")
    logger.info(f"Corresp.:     {len(result.correspondence_set)}")
    logger.info(f"Fitness:      {result.fitness}")
    logger.info(f"Inlier RMSE:  {result.inlier_rmse}")
    logger.info(f"Trans. scale:    {scale}")
    logger.info(f"Trans. trans.:   {translation}")
    logger.info(f"Trans. rot.:     {rotation}")
    logger.info("")

### Build registration modules from config

In [ ]:
from dataclasses import dataclass
from typing import TypeAlias

from mynd.io import read_config

# Processors and registrators
from mynd.registration import (
    PointCloudProcessor,
    GlobalRegistrator,
    IncrementalRegistrator,
)

# Builders
from mynd.registration import (
    build_point_cloud_processor,
    build_feature_registrator,
    build_icp_registrator,
)

from mynd.registration import RegistrationPipeline, apply_registration_pipeline


@dataclass
class RegistrationConfig:
    """Class representing a registration configuration."""

    @dataclass
    class Module:
        """Class representing a config item."""

        preprocessor: list[dict]
        registrator: dict

    initializer: Module
    incrementors: list[Module]


def create_registration_config(config: dict) -> RegistrationConfig:
    """Creates a registration pipeline config."""
    # TODO: Get initializer modules
    initializer: RegistrationConfig.Module = RegistrationConfig.Module(
        **config.get("initializer")
    )
    # TODO: Get incrementor modules
    if "incrementors" in config:
        incrementors: list[RegistrationConfig.Module] = [
            RegistrationConfig.Module(**item)
            for item in config.get("incrementors")
        ]
    else:
        incrementors: list = list()
    return RegistrationConfig(initializer, incrementors)


CONFIG_FILE: Path = Path(
    "/home/martin/dev/mynd/config/registration_simple.toml"
)
config: dict = read_config(CONFIG_FILE).unwrap()

config: RegistrationConfig = create_registration_config(
    config.get("registration")
)

logger.info("---------- Registration ----------")
logger.info(f" - Initializer: {config.initializer}")
logger.info(f" - Incrementors: {config.incrementors}")
logger.info("----------------------------------")


# TODO: Build global registrator
initializer: RegistrationPipeline.InitializerModule = (
    RegistrationPipeline.InitializerModule(
        preprocessor=build_point_cloud_processor(
            **config.initializer.preprocessor
        ),
        registrator=build_feature_registrator(
            config.initializer.registrator
        ).unwrap(),
    )
)

pipeline: RegistrationPipeline = RegistrationPipeline(initializer)

target: PointCloud = loaders.get(0)().unwrap()
source: PointCloud = loaders.get(1)().unwrap()

result: RegistrationResult = apply_registration_pipeline(
    pipeline, source=source, target=target
)

logger.info(result)

# result: RegistrationResult = initializer.registrator(target=target_pre, source=source_pre)
# apply_registration_pipeline

# TODO: Build incremental registrators
# registrator: IncrementalRegistrator = build_icp_registrator(
#    module_config["registrator"],
# ).unwrap()

# build_point_cloud_processor(
#    model["method"], model["parameters"]
# ).unwrap()

2024-11-03 12:15:24.524 | INFO     | __main__:<module>:59 - ---------- Registration ----------
2024-11-03 12:15:24.524 | INFO     | __main__:<module>:60 -  - Initializer: RegistrationConfig.Module(preprocessor={'downsample': {'spacing': 0.2}, 'estimate_normals': {'radius': 0.4}}, registrator={'algorithm': 'feature_matching_ransac', 'parameters': {'distance_threshold': 0.15, 'sample_count': 3, 'mutual_filter': True}, 'component_types': {'feature_extractor': 'fpfh', 'estimation_method': 'point_to_point', 'validators': 'correspondence_validators', 'convergence_criteria': 'ransac_convergence_criteria'}, 'component_parameters': {'feature_extractor': {'radius': 2.0, 'neighbours': 300}, 'estimation_method': {'with_scaling': True}, 'validators': {'distance_threshold': 0.15, 'edge_threshold': 0.95, 'normal_threshold': 5.0}, 'convergence_criteria': {'max_iteration': 1000, 'confidence': 1.0}}})
2024-11-03 12:15:24.524 | INFO     | __main__:<module>:61 -  - Incrementors: []
2024-11-03 12:15:24.524